<a href="https://colab.research.google.com/github/kietbg0079/delete_all_files_google_drive/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile as zp

In [ ]:
path = '/content/drive/MyDrive/AIP302/flower_data.zip'

raw_data = zp.ZipFile(path)
raw_data.extractall('/content/')

In [ ]:
import math
import random
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil 

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models,layers,optimizers
from keras.preprocessing import image
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization

#from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping,  ModelCheckpoint, LearningRateScheduler

from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input

In [ ]:
batch_size = 64
img_height = 224
img_width = 224

In [ ]:
TRAINING_DIR = '/content/flower_data/train'



train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   horizontal_flip=True,
                                   width_shift_range=0.2,
                                   shear_range=0.2,
                                   )

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))


VALIDATION_DIR = '/content/flower_data/valid'

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              target_size=(img_height, img_width))  

Found 6552 images belonging to 102 classes.
Found 818 images belonging to 102 classes.


In [ ]:
adam = Adam(learning_rate=3e-4)

resnet101_base = ResNet101(include_top=True, weights='imagenet',
                          input_shape=(img_width, img_height,3))

output = resnet101_base.get_layer(index = -1).output  
output = Flatten()(output)

output = Dense(512,activation = "relu")(output)
output = BatchNormalization()(output)
output = Dropout(0.2)(output)
output = Dense(512,activation = "relu")(output)
output = BatchNormalization()(output)
output = Dropout(0.2)(output)
output = Dense(102, activation='softmax')(output)

resnet101_model = Model(resnet101_base.input, output)
for layer in resnet101_model.layers[:-7]:
    layer.trainable = False
resnet101_model.summary()

resnet101_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
resnet101_model.load_weights('/content/drive/MyDrive/AIL201/AIP302/Landmark_weights_epoch-53_loss-1.1591_val_loss-1.4267.h5')

In [ ]:
epoch = 100

history = resnet101_model.fit(train_generator,
                              epochs=epoch,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [ModelCheckpoint('Landmark_weights_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                                          monitor='val_loss',
                                                          verbose=1,
                                                          save_best_only=True,
                                                          save_weights_only=True,
                                                          mode='auto',
                                                          period=1),
                                          EarlyStopping(monitor='val_loss',
                                                        patience=7, 
                                                        verbose=1, 
                                                        min_delta=0.001),
                              ])

In [ ]:
shutil.move('/content/Landmark_weights_epoch-53_loss-1.1591_val_loss-1.4267.h5', '/content/drive/MyDrive/AIP302')

'/content/drive/MyDrive/AIP302/Landmark_weights_epoch-53_loss-1.1591_val_loss-1.4267.h5'